In [131]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [132]:
age_by_sex = df.groupby(['Sex','Pclass','Embarked'])['Age'].mean().reset_index()

age_by_sex = age_by_sex.rename(columns={'Age': 'Mean_Age'})

df = df.merge(age_by_sex)

df['Age'] = df['Age'].fillna(df['Mean_Age'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        202 non-null    object 
 11  Embarked     889 non-null    object 
 12  Mean_Age     889 non-null    float64
dtypes: float64(3), int64(5), object(5)
memory usage: 90.4+ KB


In [133]:
df['Age_decade'] = df['Age'] - df['Age'] % 10
df['Title'] = df['Name'].str.extract(r'(\s[^\.]+)')
df['Family_size'] = df['SibSp'] + df['Parch'] + 1
df['Sex'] = df['Sex'].replace('male', 1)
df['Sex'] = df['Sex'].replace('female', 0)
df['Fare'] = df['Fare'] - df['Fare'] % 10
df['Embarked'] = df['Embarked'].replace('S', 0)
df['Embarked'] = df['Embarked'].replace('C', 1)
df['Embarked'] = df['Embarked'].replace('Q', 3)

df.head()

/tmp/ipykernel_9862/1936003436.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'] = df['Sex'].replace('female', 0)
/tmp/ipykernel_9862/1936003436.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Embarked'] = df['Embarked'].replace('Q', 3)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Mean_Age,Age_decade,Title,Family_size
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,0.0,NaN,0,26.574766,20.0,Mr,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,70.0,C85,1,36.052632,30.0,Mrs,2
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,0.0,NaN,0,23.223684,20.0,Miss,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,50.0,C123,0,32.704545,30.0,Mrs,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,0.0,NaN,0,26.574766,30.0,Mr,1


In [134]:
df.drop(columns=['PassengerId', 'Name','Mean_Age', 'Age', 'SibSp', 'Parch', 'Cabin', 'Ticket', 'Title'])

,Survived,Pclass,Sex,Fare,Embarked,Age_decade,Family_size
0,0,3,1,0.0,0,20.0,2
1,1,1,0,70.0,1,30.0,2
2,1,3,0,0.0,0,20.0,1
3,1,1,0,50.0,0,30.0,2
4,0,3,1,0.0,0,30.0,1
...,...,...,...,...,...,...,...
884,0,2,1,10.0,0,20.0,1
885,1,1,0,30.0,0,10.0,1
886,0,3,0,20.0,0,20.0,4
887,1,1,1,30.0,1,20.0,1


In [150]:
X = df[['Pclass', 'Sex', 'Fare', 'Age_decade', 'Family_size', 'Embarked']]
y = df['Survived']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
( cnf_matrix[0][0] + cnf_matrix[1][1] ) / (cnf_matrix[0][1]+cnf_matrix[0][0]+cnf_matrix[1][1]+cnf_matrix[1][0])

( cnf_matrix[0][0]) / (cnf_matrix[0][1]+cnf_matrix[0][0])



np.float64(0.8888888888888888)

In [151]:
from sklearn.metrics import classification_report

target_names = ['Not survived', 'Survived']

print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

Not survived       0.78      0.89      0.83       126
    Survived       0.82      0.68      0.75        97

    accuracy                           0.80       223
   macro avg       0.80      0.78      0.79       223
weighted avg       0.80      0.80      0.79       223

